In [ ]:
import zarr
import numpy as np
import matplotlib.pyplot as plt
import torch
import pickle
import os
import seaborn as sns; sns.set()  # for plot styling
from matplotlib.colors import BoundaryNorm
import xarray as xr
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.patches as mpatches
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import random
from functions1 import *
from matplotlib.colors import BoundaryNorm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from torchdataloaders import TestDataset,get_variable_options, TestDatasetPermute
from utils1 import CHARTS, SIC_LOOKUP, SOD_LOOKUP, FLOE_LOOKUP, SCENE_VARIABLES, colour_str


os.environ['AI4ARCTIC_DATA'] = '/work3/s222651/data/data/dataV2/'
os.environ['AI4ARCTIC_ENV'] = '/work3/s222651/'
train_options = {
    # -- Training options -- #
    'path_to_processed_data': os.environ['AI4ARCTIC_DATA'], 
    'test_files': '/work3/s222651/test/data/testdata/', 
    'path_to_env': os.environ['AI4ARCTIC_ENV'],  
    'lr': 0.0001,  # Optimizer learning rate.
    'epochs': 70,  # Number of epochs before training stop.
    'epoch_len': 500,  # Number of batches for each epoch.
    'patch_size': 256,  # Size of patches sampled. Used for both Width and Height.
    'batch_size': 8,  # Number of patches for each batch.
    'loader_upsampling': 'nearest',  # How to upscale low resolution variables to high resolution.
    
    # -- Data prepraration lookups and metrics.
    'train_variables': SCENE_VARIABLES,  # Contains the relevant variables in the scenes.
    'charts': CHARTS,  # Charts to train on.
    'n_classes': {  # number of total classes in the reference charts, including the mask.
        'SIC': SIC_LOOKUP['n_classes'],
        'SOD': SOD_LOOKUP['n_classes'],
        'FLOE': FLOE_LOOKUP['n_classes']
    },
    'pixel_spacing': 80,  # SAR pixel spacing. 80 for the ready-to-train AI4Arctic Challenge dataset.
    'train_fill_value': 0,  # Mask value for SAR training data.
    'class_fill_values': {  # Mask value for class/reference data.
        'SIC': SIC_LOOKUP['mask'],
        'SOD': SOD_LOOKUP['mask'],
        'FLOE': FLOE_LOOKUP['mask'],
    },
    
    # -- Validation options -- #
    'chart_metric': {  # Metric functions for each ice parameter and the associated weight.
        #'SIC': {
        #    'func': r2_metric,
        #   'weight': 2,
        #},
        'SOD': {
            'func': f1_metric,
        #    'weight': 2,
        },
        #'FLOE': {
        #    'func': f1_metric,
        #    'weight': 1,
        
    },
    'num_val_scenes': 20,  # Number of scenes randomly sampled from train_list to use in validation.
    
    # -- GPU/cuda options -- #
    'gpu_id': 0,  # Index of GPU. In case of multiple GPUs.
    'num_workers': 8,  # Number of parallel processes to fetch data.
    'num_workers_val': 1,  # Number of parallel processes during validation.
    
    # -- U-Net Options -- #
    'unet_conv_filters': [16, 32, 64, 64],  # Number of filters in the U-Net.
    'conv_kernel_size': (3, 3),  # Size of convolutional kernels.
    'conv_stride_rate': (1, 1),  # Stride rate of convolutional kernels.
    'conv_dilation_rate': (1, 1),  # Dilation rate of convolutional kernels.
    'conv_padding': (1, 1),  # Number of padded pixels in convolutional layers.
    'conv_padding_style': 'zeros',  # Style of padding.
}
# Get options for variables, amsrenv grid, cropping and upsampling.
get_variable_options = get_variable_options(train_options)

### inspection of training data

In [ ]:
# open nc file 
#ds = xr.open_dataset('C:\\Users\\HpRyzen7\\Desktop\\Thesis\\AI4Arctic\\dataV2\\20211220T102441_dmi_prep.nc') #this is ready to train version
ds = xr.open_dataset('C:\\Users\\HpRyzen7\\Desktop\\Thesis\\AI4Arctic\\testdata\\20210506T075557_dmi_prep.nc') 
#ds = xr.open_dataset('C:\\Users\\HpRyzen7\\Desktop\\Thesis\\AI4Arctic\\raw\\S1A_EW_GRDM_1SDH_20190406T102029_20190406T102058_026668_02FE1F_C66E_icechart_cis_SGRDINFLD_20190406T1019Z_pl_a.nc') # this is the original version
HH = ds['nersc_sar_primary']
HV = ds['nersc_sar_secondary']
bt_6_9h = ds['btemp_6_9h']
bt_6_9v = ds['btemp_6_9v']
bt_7_3v = ds['btemp_7_3v']
bt_7_3h = ds['btemp_7_3h']
bt_10_7h = ds['btemp_10_7h']
bt_10_7v = ds['btemp_10_7v']
bt_18_7h = ds['btemp_18_7h']
bt_18_7v = ds['btemp_18_7v']    
bt_23_8h = ds['btemp_23_8h']
bt_23_8v = ds['btemp_23_8v']
bt_36_5h = ds['btemp_36_5h']
bt_36_5v = ds['btemp_36_5v']
bt_89_0h = ds['btemp_89_0h']
bt_89_0v = ds['btemp_89_0v']
#sodlabel = ds['SOD']


# plot the HH and SOD label in a figure with their colorbars 
fig, ax = plt.subplots(1, 2, figsize=(16, 5))
im0 = ax[0].imshow(HH, cmap='gray')
ax[0].set_title('SAR - HV')
ax[0].grid(False)
ax[0].set_xticks([])
ax[0].set_yticks([])
divider = make_axes_locatable(ax[0])
cax = divider.append_axes("right", size="5%", pad=0.05)
cbar0 = plt.colorbar(im0, cax=cax)
cbar0.ax.set_title(r'$\sigma^0 \ [dB]$', loc='center', fontsize=10)

im1 = ax[1].imshow(bt_36_5v, cmap='viridis')
ax[1].set_title('36.5 GHz - V polarization')
ax[1].grid(False)
ax[1].set_xticks([])
ax[1].set_yticks([])
divider = make_axes_locatable(ax[1])
cax = divider.append_axes("right", size="5%", pad=0.05)
cbar1 = plt.colorbar(im1, cax=cax)
cbar1.ax.set_title('Bt', loc='center', fontsize=10)

In [ ]:
# load the training nc files to count class occurences

directory = 'C:\\Users\\HpRyzen7\\Desktop\\Thesis\\AI4Arctic\\dataV2\\'

# List to store the counts of each class
class_counts = np.zeros(6)  

# Total valid pixels counter (excluding 255)
total_valid_pixels = 0
for filename in os.listdir(directory):
    if filename.endswith('.nc'):
        # Load the dataset
        ds = xr.open_dataset(os.path.join(directory, filename))
        
        # Extract the 'SOD' variable
        sod = ds['SOD'].values
        
        # Count each class ignoring 255
        unique, counts = np.unique(sod[sod != 255], return_counts=True)
        #unique, counts = np.unique(sod, return_counts=True)
        
        # Update class counts and total valid pixels
        for i, val in enumerate(unique):
            if val in range(6):  # Ensure class value is within 0-5
                class_counts[int(val)] += counts[i]
        total_valid_pixels += np.sum(counts)

        # Close the dataset to free resources
        ds.close()

percentages = (class_counts / total_valid_pixels) * 100

for i, pct in enumerate(percentages):
    print(f"Class {i}: {pct:.2f}%")

In [ ]:
# histogram of the class occurences
classes = ['Open-water', 'New ice', 'Young ice ', 'Thin FYI', 'Thick FYI', 'Old ice']

plt.figure(figsize=(8, 6))
plt.bar(classes, percentages)
plt.xlabel('Class')
plt.ylabel('Percentage [%]')
plt.title('SOD class distribution')
for i, pct in enumerate(percentages):
    plt.text(i, pct, f"{pct:.2f}%", ha='center', va='bottom', fontsize=9)
plt.grid(False)
plt.show()


### Permutation visualized

In [ ]:
seed=42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

bt = bt
flattened_bt = bt.flatten()
perm_indices = torch.randperm(flattened_bt.shape[0])
bt_permuted = flattened_bt[perm_indices].reshape(bt.shape)

# Plotting the original and permuted 36.5 GHz V band
fig, ax = plt.subplots(1, 2, figsize=(10, 5))

# Original data
im0 = ax[0].imshow(bt_36_5h, cmap='viridis')
ax[0].set_title('36.5 GHz - V polarization')
ax[0].grid(False)
ax[0].set_xticks([])
ax[0].set_yticks([])
divider = make_axes_locatable(ax[0])
cax = divider.append_axes("right", size="5%", pad=0.05)
cbar0 = plt.colorbar(im0, cax=cax)
cbar0.ax.set_title('Bt', loc='center', fontsize=10)

# Permuted data
im1 = ax[1].imshow(bt_permuted, cmap='viridis')
ax[1].set_title('36.5 GHz - V polarization - Permuted')
ax[1].grid(False)
ax[1].set_xticks([])
ax[1].set_yticks([])
divider = make_axes_locatable(ax[1])
cax = divider.append_axes("right", size="5%", pad=0.05)
cbar1 = plt.colorbar(im1, cax=cax)
cbar1.ax.set_title('Bt', loc='center', fontsize=10)

plt.show()

In [ ]:
# locate the VALIDATION files
#val_dir = 'C:\\Users\\HpRyzen7\\Desktop\\Thesis\\AI4Arctic\\validationdata\\'

test_dir = 'C:\\Users\\HpRyzen7\\Desktop\\Thesis\\AI4Arctic\\testdata\\'

# import nc files 
# validation data

testfile1 = test_dir + '20180124T194759_dmi_prep.nc'
testfile2 = test_dir + '20180623T114935_cis_prep.nc'
testfile3 = test_dir + '20180707T113313_cis_prep.nc'
testfile4 = test_dir + '20180716T110418_cis_prep.nc'
testfile5 = test_dir + '20180903T123331_cis_prep.nc'
testfile6 = test_dir + '20180917T121813_cis_prep.nc'
testfile7 = test_dir + '20190406T102029_cis_prep.nc'
testfile8 = test_dir + '20190810T110422_dmi_prep.nc'
testfile9 = test_dir + '20191011T131651_cis_prep.nc'
testfile10 = test_dir + '20200217T102731_cis_prep.nc'
testfile11 = test_dir + '20200319T101935_cis_prep.nc'
testfile12 = test_dir + '20200701T114012_cis_prep.nc'
testfile13 = test_dir + '20200719T123046_cis_prep.nc'
testfile14 = test_dir + '20201013T080448_dmi_prep.nc'
testfile15 = test_dir + '20210328T202742_dmi_prep.nc'
testfile16 = test_dir + '20210410T201933_dmi_prep.nc'
testfile17 = test_dir + '20210430T205436_dmi_prep.nc'
testfile18 = test_dir + '20210506T075557_dmi_prep.nc'
testfile19 = test_dir + '20210512T214149_cis_prep.nc'
testfile20 = test_dir + '20211212T211242_dmi_prep.nc'

testds1 = xr.open_dataset(testfile1)
testds2 = xr.open_dataset(testfile2)
testds3 = xr.open_dataset(testfile3)
testds4 = xr.open_dataset(testfile4)
testds5 = xr.open_dataset(testfile5)
testds6 = xr.open_dataset(testfile6)
testds7 = xr.open_dataset(testfile7)
testds8 = xr.open_dataset(testfile8)
testds9 = xr.open_dataset(testfile9)
testds10 = xr.open_dataset(testfile10)
testds11 = xr.open_dataset(testfile11)
testds12 = xr.open_dataset(testfile12)
testds13 = xr.open_dataset(testfile13)
testds14 = xr.open_dataset(testfile14)
testds15 = xr.open_dataset(testfile15)
testds16 = xr.open_dataset(testfile16)
testds17 = xr.open_dataset(testfile17)
testds18 = xr.open_dataset(testfile18)
testds19 = xr.open_dataset(testfile19)
testds20 = xr.open_dataset(testfile20)

# plot the longitude and latitude of the test data in the Arctic region in one plot (polar stereographic projection)
fig, ax = plt.subplots(subplot_kw={'projection': ccrs.NorthPolarStereo()}, figsize=(7, 7))
ax.set_extent([-150, 150, 50, 90], ccrs.PlateCarree())
ax.add_feature(cfeature.LAND,facecolor='lightgray', edgecolor='black')
ax.add_feature(cfeature.COASTLINE, linewidth=0.8)
ax.add_feature(cfeature.BORDERS, linewidth=0.5)
ax.gridlines()
ax.set_title('Test scenes')
ax.plot(testds1['sar_grid2d_longitude'].values, testds1['sar_grid2d_latitude'].values, color = 'red', linewidth = 0.5, transform=ccrs.PlateCarree())
ax.plot(testds2['sar_grid2d_longitude'].values, testds2['sar_grid2d_latitude'].values, color = 'red', linewidth = 0.5, transform=ccrs.PlateCarree())
ax.plot(testds3['sar_grid2d_longitude'].values, testds3['sar_grid2d_latitude'].values, color = 'red', linewidth = 0.5, transform=ccrs.PlateCarree())
ax.plot(testds4['sar_grid2d_longitude'].values, testds4['sar_grid2d_latitude'].values, color = 'red', linewidth = 0.5, transform=ccrs.PlateCarree())
ax.plot(testds5['sar_grid2d_longitude'].values, testds5['sar_grid2d_latitude'].values, color = 'red', linewidth = 0.5, transform=ccrs.PlateCarree())
ax.plot(testds6['sar_grid2d_longitude'].values, testds6['sar_grid2d_latitude'].values, color = 'red', linewidth = 0.5, transform=ccrs.PlateCarree())
ax.plot(testds7['sar_grid2d_longitude'].values, testds7['sar_grid2d_latitude'].values, color = 'red', linewidth = 0.5, transform=ccrs.PlateCarree())
ax.plot(testds8['sar_grid2d_longitude'].values, testds8['sar_grid2d_latitude'].values, color = 'red', linewidth = 0.5, transform=ccrs.PlateCarree())
ax.plot(testds9['sar_grid2d_longitude'].values, testds9['sar_grid2d_latitude'].values, color = 'red', linewidth = 0.5, transform=ccrs.PlateCarree())
ax.plot(testds10['sar_grid2d_longitude'].values, testds10['sar_grid2d_latitude'].values, color = 'red', linewidth = 0.5, transform=ccrs.PlateCarree())
ax.plot(testds11['sar_grid2d_longitude'].values, testds11['sar_grid2d_latitude'].values, color = 'red', linewidth = 0.5, transform=ccrs.PlateCarree())
ax.plot(testds12['sar_grid2d_longitude'].values, testds12['sar_grid2d_latitude'].values, color = 'red', linewidth = 0.5, transform=ccrs.PlateCarree())
ax.plot(testds13['sar_grid2d_longitude'].values, testds13['sar_grid2d_latitude'].values, color = 'red', linewidth = 0.5, transform=ccrs.PlateCarree())
ax.plot(testds14['sar_grid2d_longitude'].values, testds14['sar_grid2d_latitude'].values, color = 'red', linewidth = 0.5, transform=ccrs.PlateCarree())
ax.plot(testds15['sar_grid2d_longitude'].values, testds15['sar_grid2d_latitude'].values, color = 'red', linewidth = 0.5, transform=ccrs.PlateCarree())
ax.plot(testds16['sar_grid2d_longitude'].values, testds16['sar_grid2d_latitude'].values, color = 'red', linewidth = 0.5, transform=ccrs.PlateCarree())
ax.plot(testds17['sar_grid2d_longitude'].values, testds17['sar_grid2d_latitude'].values, color = 'red', linewidth = 0.5, transform=ccrs.PlateCarree())
ax.plot(testds18['sar_grid2d_longitude'].values, testds18['sar_grid2d_latitude'].values, color = 'red', linewidth = 0.5, transform=ccrs.PlateCarree())
ax.plot(testds19['sar_grid2d_longitude'].values, testds19['sar_grid2d_latitude'].values, color = 'red', linewidth = 0.5, transform=ccrs.PlateCarree())
ax.plot(testds20['sar_grid2d_longitude'].values, testds20['sar_grid2d_latitude'].values, color = 'red', linewidth = 0.5, transform=ccrs.PlateCarree())
plt.show()


### 70 epochs and 90 epochs with all channels - performance metrics

In [ ]:
model70 = 'C:\\Users\\HpRyzen7\\Desktop\\Thesis\\AI4Arctic\\test_results\\inference_70_epochs_16channels/best_model.pth'
#model90 = 'C:\\Users\\HpRyzen7\\Desktop\\Thesis\\AI4Arctic\\test_results\\inference_90_epochs_16channels/best_model.pth'

checkpoint70 = torch.load(model70, map_location='cpu')
#checkpoint90 = torch.load(model90, map_location='cpu')


best_F1_score70 = checkpoint70['best_score']
#best_F1_score90 = checkpoint90['best_score']

print(f"best F1 score (70 epochs and all channels): {best_F1_score70} %")
#print(f"best F1 score (90 epochs and all channels): {best_F1_score90} %")

# plot the two confusion matrices side by side
cm70 = np.load('C:\\Users\\HpRyzen7\\Desktop\\Thesis\\AI4Arctic\\test_results\\inference_70_epochs_16channels//confusion_matrix.npy')
cm_percentage70 = cm70 / cm70.sum(axis=1, keepdims=True) * 100

#cm90 = np.load('C:\\Users\\HpRyzen7\\Desktop\\Thesis\\AI4Arctic\\test_results\\inference_90_epochs_16channels//confusion_matrix.npy')
#cm_percentage90 = cm90 / cm90.sum(axis=1, keepdims=True) * 100

fig, ax = plt.subplots(1, 2, figsize=(14, 5))
sns.heatmap(cm_percentage70, annot=True, fmt=".1f", cmap='YlGnBu', cbar_kws={'label': 'Percentage of Predictions [%]'}, ax=ax[0])
ax[0].set_xlabel('Predicted')
ax[0].set_ylabel('True')
ax[0].set_title('70 epochs all channels')

# sns.heatmap(cm_percentage90, annot=True, fmt=".1f", cmap='YlGnBu', cbar_kws={'label': 'Percentage of Predictions [%]'}, ax=ax[1])
# ax[1].set_xlabel('Predicted')
# ax[1].set_ylabel('True')
# ax[1].set_title('90 epochs all channels')

plt.show()

### Loading the predictions and reference to do plots

In [ ]:
# reference

pathref = 'C:\\Users\\HpRyzen7\\Desktop\\Thesis\\AI4Arctic\\testreference\\'
scene_name = '20201013T080448_dmi_prep_reference.nc'

ref = xr.open_dataset(pathref + scene_name)
ref_sod = ref['SOD']
print(ref_sod.shape)

# replace the 255 values in the sod variable with nan
ref_sod = ref_sod.where(ref_sod != 255, np.nan)


path = 'C:\\Users\\HpRyzen7\\Desktop\\Thesis\\AI4Arctic\\test_results\\inference_70_epochs_16channels\\'
scene_name = '20201013T080448_dmi_prep'

# load npz file 
pred = np.load(path + scene_name + '_data.npz')

pred_sod = pred['SOD_pred']
pred_masks = pred['test_mask']

print(pred_sod.shape)

boundaries = np.arange(-0.5, 6, 1)
cmap = plt.cm.get_cmap('viridis', len(boundaries) - 1)
norm = BoundaryNorm(boundaries, cmap.N)


fig, axs = plt.subplots(1, 2, figsize=(10, 8))
# plot SOD prediction vs reference
im_pred = axs[0].imshow(pred_sod[...], cmap=cmap, norm=norm)
axs[0].set_title('Prediction ')
axs[0].grid(False)

for spine in axs[0].spines.values():
    spine.set_edgecolor('black')
    spine.set_linewidth(1)

im_ref = axs[1].imshow(ref_sod[...], cmap=cmap, norm=norm)
axs[1].set_title('Truth')
axs[1].grid(False)

for spine in axs[1].spines.values():
    spine.set_edgecolor('black')
    spine.set_linewidth(1)

cbar = plt.colorbar(im_pred, ax=axs, orientation = 'horizontal', fraction = 0.08, pad = 0.07, ticks = np.arange(0,len(boundaries)-1))

class_labels = ['Open water', 'New ice', 'Young ice', 'Thin FYI', 'Thick FYI', 'Multiyear ice']
cbar.set_ticklabels(class_labels)
cbar.set_label('Stage of Development')
plt.show()

In [ ]:
model1 = 'C:\\Users\\HpRyzen7\\Desktop\\Thesis\\AI4Arctic\\test_results\\inference_70_epochs_16channels/best_model.pth'
model2 = 'C:\\Users\\HpRyzen7\\Desktop\\Thesis\\AI4Arctic\\test_results\\inference_70_epochs_16channelsNEW/best_model.pth'
model3 = 'C:\\Users\\HpRyzen7\\Desktop\\Thesis\\AI4Arctic\\test_results\\inference_70_epochs_16channelsNEWNEW/best_model.pth'
model4 = 'C:\\Users\\HpRyzen7\\Desktop\\Thesis\\AI4Arctic\\test_results\\model4/best_model.pth'
model5 = 'C:\\Users\\HpRyzen7\\Desktop\\Thesis\\AI4Arctic\\test_results\\model5/best_model.pth'
model6 = 'C:\\Users\\HpRyzen7\\Desktop\\Thesis\\AI4Arctic\\test_results\\model6/best_model.pth'
model7 = 'C:\\Users\\HpRyzen7\\Desktop\\Thesis\\AI4Arctic\\test_results\\model7/best_model.pth'
model8 = 'C:\\Users\\HpRyzen7\\Desktop\\Thesis\\AI4Arctic\\test_results\\model8/best_model.pth'
model9 = 'C:\\Users\\HpRyzen7\\Desktop\\Thesis\\AI4Arctic\\test_results\\model9/best_model.pth'
model10 = 'C:\\Users\\HpRyzen7\\Desktop\\Thesis\\AI4Arctic\\test_results\\model10/best_model.pth'
##
checkpoint1 = torch.load(model1, map_location='cpu')
checkpoint2 = torch.load(model2, map_location='cpu')
checkpoint3 = torch.load(model3, map_location='cpu')
checkpoint4 = torch.load(model4, map_location='cpu')
checkpoint5 = torch.load(model5, map_location='cpu')
checkpoint6 = torch.load(model6, map_location='cpu')
checkpoint7 = torch.load(model7, map_location='cpu')
checkpoint8 = torch.load(model8, map_location='cpu')
checkpoint9 = torch.load(model9, map_location='cpu')
checkpoint10 = torch.load(model10, map_location='cpu')
##
best_F1_score1 = checkpoint1['best_score']
best_F1_score2 = checkpoint2['best_score']
best_F1_score3 = checkpoint3['best_score']
best_F1_score4 = checkpoint4['best_score']
best_F1_score5 = checkpoint5['best_score']
best_F1_score6 = checkpoint6['best_score']
best_F1_score7 = checkpoint7['best_score']
best_F1_score8 = checkpoint8['best_score']
best_F1_score9 = checkpoint9['best_score']
best_F1_score10 = checkpoint10['best_score']
##
print(f"best F1 score (model1): {best_F1_score1} %")
print(f"best F1 score (model2): {best_F1_score2} %")
print(f"best F1 score (model3): {best_F1_score3} %")
print(f"best F1 score (model4): {best_F1_score4} %")
print(f"best F1 score (model5): {best_F1_score5} %")
print(f"best F1 score (model6): {best_F1_score6} %")
print(f"best F1 score (model7): {best_F1_score7} %")
print(f"best F1 score (model8): {best_F1_score8} %")
print(f"best F1 score (model9): {best_F1_score9} %")
print(f"best F1 score (model10): {best_F1_score10} %")

# std of the F1 scores for the 10 models
F1_scores = [best_F1_score1, best_F1_score2, best_F1_score3, best_F1_score4, best_F1_score5, best_F1_score6, best_F1_score7, best_F1_score8, best_F1_score9, best_F1_score10]

std_F1_scores = np.std(F1_scores)
mean_F1_scores = np.mean(F1_scores)

# print mean and std with 2 decimal places
print(f"Mean F1 score: {mean_F1_scores:.2f} %")
print(f"Std F1 score: {std_F1_scores:.2f} %")

In [ ]:
cm = np.load('C:\\Users\\HpRyzen7\\Desktop\\Thesis\\AI4Arctic\\test_results\\model4\\confusion_matrix.npy')
cm_percentage = cm / cm.sum(axis=1, keepdims=True) * 100

fig, ax = plt.subplots(1, 2, figsize=(14, 5))
sns.heatmap(cm_percentage, annot=True, fmt=".1f", cmap='YlGnBu', cbar_kws={'label': 'Percentage of Predictions [%]'}, ax=ax[0])
ax[0].set_xlabel('Predicted')
ax[0].set_ylabel('True')
ax[0].set_title('Confusion matrix')

plt.show()

### Experiment 1: Excluding important channels 18.7 V / 23.8 H / 36.5 V & H

In [ ]:
# experiment 1: excluding important channels 18.7 V / 23.8 H  / 36.5 V & H

model_exp1 = 'C:\\Users\\HpRyzen7\\Desktop\\Thesis\\AI4Arctic\\test_results\\exp1\\best_model.pth'
checkpoint_exp1 = torch.load(model_exp1, map_location='cpu')
best_F1_score_exp1 = checkpoint_exp1['best_score']
print(f"best F1 score (experiment 1): {best_F1_score_exp1} %")

# it is out out of the std deviation of the 10 models !


cm = np.load('C:\\Users\\HpRyzen7\\Desktop\\Thesis\\AI4Arctic\\test_results\\exp1\\confusion_matrix.npy')
cm_percentage = cm / cm.sum(axis=1, keepdims=True) * 100

fig, ax = plt.subplots(1, 2, figsize=(14, 5))
sns.heatmap(cm_percentage, annot=True, fmt=".1f", cmap='YlGnBu', cbar_kws={'label': 'Percentage of Predictions [%]'}, ax=ax[0])
ax[0].set_xlabel('Predicted')
ax[0].set_ylabel('True')
ax[0].set_title('Confusion matrix')

plt.show()

### inference from the experiment 1

In [ ]:

# load the label

ds = xr.open_dataset('C:\\Users\\HpRyzen7\\Desktop\\Thesis\\AI4Arctic\\testreference\\20200319T101935_cis_prep_reference.nc') 

sod = ds['SOD']

# whatever is 255, it is nan
sod = sod.where(sod != 255)

sod_flip = np.flipud(sod)


for chart in train_options['charts']:

    #plot the sod 
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    im = ax.imshow(sod_flip, vmin=0, vmax = train_options['n_classes'][chart] -2, cmap ='jet', interpolation = 'nearest')
    ax.set_title('SOD')
    chart_cbar(ax=ax,n_classes=train_options['n_classes'][chart], chart=chart, cmap='jet')
    # grid off
    ax.grid(False)
    plt.show()

### Experiment 2: Excluding very low and very high frequencies  6.9 / 7.3 / 89.0

In [ ]:
# experiment 2: excluding very low and very high frequencies 6.9 / 7.3 / 89.0 both polarizations
model_exp2 = 'C:\\Users\\HpRyzen7\\Desktop\\Thesis\\AI4Arctic\\test_results\\exp2/best_model.pth'
checkpoint_exp2 = torch.load(model_exp2, map_location='cpu')
best_F1_score_exp2 = checkpoint_exp2['best_score']
print(f"best F1 score (experiment 2): {best_F1_score_exp2} %")

# it is out out of the std deviation of the 10 models !


cm = np.load('C:\\Users\\HpRyzen7\\Desktop\\Thesis\\AI4Arctic\\test_results\\exp2\\confusion_matrix.npy')
cm_percentage = cm / cm.sum(axis=1, keepdims=True) * 100

fig, ax = plt.subplots(1, 2, figsize=(14, 5))
sns.heatmap(cm_percentage, annot=True, fmt=".1f", cmap='YlGnBu', cbar_kws={'label': 'Percentage of Predictions [%]'}, ax=ax[0])
ax[0].set_xlabel('Predicted')
ax[0].set_ylabel('True')
ax[0].set_title('Confusion matrix')

plt.show()

### Experiment 3: Simulation of CIMR --  6.9 / 10.65 / 18.7 / 36.5 

In [ ]:
# experiment 2: excluding very low and very high frequencies 6.9 / 7.3 / 89.0 both polarizations
model_exp3 = 'C:\\Users\\HpRyzen7\\Desktop\\Thesis\\AI4Arctic\\test_results\\cimr/best_model.pth'
checkpoint_exp3 = torch.load(model_exp3, map_location='cpu')
best_F1_score_exp3 = checkpoint_exp3['best_score']
print(f"best F1 score (experiment 3): {best_F1_score_exp3} %")


# print the model details
#print(checkpoint_exp3)

cm = np.load('C:\\Users\\HpRyzen7\\Desktop\\Thesis\\AI4Arctic\\test_results\\cimr\\confusion_matrix.npy')
cm_percentage = cm / cm.sum(axis=1, keepdims=True) * 100

fig, ax = plt.subplots(1, 2, figsize=(14, 5))
sns.heatmap(cm_percentage, annot=True, fmt=".1f", cmap='YlGnBu', cbar_kws={'label': 'Percentage of Predictions [%]'}, ax=ax[0])
ax[0].set_xlabel('Predicted')
ax[0].set_ylabel('True')
ax[0].set_title('Confusion matrix')

plt.show()

### Not officially implemented from now on

### Experiment 2 - only H polarization  (10.65 H, 18.7 H, 23.5 H, 36.5 H )

In [ ]:
model_exp2_onlyH = 'C:\\Users\\HpRyzen7\\Desktop\\Thesis\\AI4Arctic\\test_results\\exp2_onlyH\\best_model.pth'
checkpoint_exp4 = torch.load(model_exp2_onlyH, map_location='cpu')
best_F1_score_exp4 = checkpoint_exp4['best_score']
print(f"best F1 score (experiment 2 only H pol): {best_F1_score_exp4} %")

# it is out out of the std deviation of the 10 models !


cm = np.load('C:\\Users\\HpRyzen7\\Desktop\\Thesis\\AI4Arctic\\test_results\\exp2_onlyH\\confusion_matrix.npy')
cm_percentage = cm / cm.sum(axis=1, keepdims=True) * 100

fig, ax = plt.subplots(1, 2, figsize=(14, 5))
sns.heatmap(cm_percentage, annot=True, fmt=".1f", cmap='YlGnBu', cbar_kws={'label': 'Percentage of Predictions [%]'}, ax=ax[0])
ax[0].set_xlabel('Predicted')
ax[0].set_ylabel('True')
ax[0].set_title('Confusion matrix')

plt.show()

### Experiment 2 - only V polarization  (10.65 V, 18.7 V, 23.5 V, 36.5 V)

In [ ]:
model_exp2_onlyV = 'C:\\Users\\HpRyzen7\\Desktop\\Thesis\\AI4Arctic\\test_results\\exp2_onlyV\\best_model.pth'
checkpoint_exp5 = torch.load(model_exp2_onlyV, map_location='cpu')
best_F1_score_exp5 = checkpoint_exp5['best_score']
print(f"best F1 score (experiment 2 only v pol): {best_F1_score_exp5} %")

# it is out out of the std deviation of the 10 models !


cm = np.load('C:\\Users\\HpRyzen7\\Desktop\\Thesis\\AI4Arctic\\test_results\\exp2_onlyV\\confusion_matrix.npy')
cm_percentage = cm / cm.sum(axis=1, keepdims=True) * 100

fig, ax = plt.subplots(1, 2, figsize=(14, 5))
sns.heatmap(cm_percentage, annot=True, fmt=".1f", cmap='YlGnBu', cbar_kws={'label': 'Percentage of Predictions [%]'}, ax=ax[0])
ax[0].set_xlabel('Predicted')
ax[0].set_ylabel('True')
ax[0].set_title('Confusion matrix')

plt.show()